# Understanding Memory in LLMs

In the previous Notebook 03, we successfully explored how OpenAI models can enhance the results from Azure Cognitive Search. [Bing Chat](http://chat.bing.com/) is a search engine with a GPT-4 model that utilizes the content of search results to provide context and deliver accurate responses to queries.

However, we have yet to discover how to engage in a conversation with the LLM. With Bing Chat, this is possible, as the LLM can understand and reference the previous responses.

There is a common misconception that GPT models have memory. This is not true. While they possess knowledge, they do not retain information from previous questions asked to them.

The aim of this Notebook is to demonstrate how we can "provide memory" to the LLM by utilizing prompts and context.

In [20]:
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferMemory
from openai.error import OpenAIError
from langchain.docstore.document import Document
from langchain.memory import CosmosDBChatMessageHistory

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

#custom libraries that we will use later in the app
from app.utils import (
    get_search_results,
    order_search_results,
    model_tokens_limit,
    num_tokens_from_docs,
    embed_docs,
    search_docs,
    get_answer,
)

from app.prompts import COMBINE_QUESTION_PROMPT, COMBINE_PROMPT, COMBINE_CHAT_PROMPT

import logging

# Get the root logger
logger = logging.getLogger()
# Set the logging level to a higher level to ignore INFO messages
logger.setLevel(logging.WARNING)

# Don't mess with this unless you really know what you are doing
AZURE_SEARCH_API_VERSION = '2021-04-30-Preview'
AZURE_OPENAI_API_VERSION = "2023-03-15-preview"


# Change these below with your own services credentials
AZURE_SEARCH_ENDPOINT = "https://cog-search-7db5ll6xo5qsy.search.windows.net"
AZURE_SEARCH_KEY = "lX5h2arV62S73XhFoGx4JG9q2lzp4wRLDuZpzk3wyXAzSeDoiXfK" # Make sure is the MANAGEMENT KEY no the query key
AZURE_OPENAI_ENDPOINT = "https://hcc-openai-instance.openai.azure.com/"
AZURE_OPENAI_API_KEY = "d879429f975f4a009008292b3e9297e4"
AZURE_COSMOSDB_ENDPOINT = "https://cosmosdb-account-7db5ll6xo5qsy.documents.azure.com:443/"
AZURE_COSMOSDB_NAME = "cosmosdb-account-7db5ll6xo5qsy"
AZURE_COSMOSDB_CONTAINER_NAME = "cosmosdb-db-7db5ll6xo5qsy"
AZURE_COMOSDB_CONNECTION_STRING = "AccountEndpoint=https://cosmosdb-account-7db5ll6xo5qsy.documents.azure.com:443/;AccountKey=o2qwd3q4OeqC0J5D02VUkIkrCUbmxvGz7mStlWgH3f12al7OFeDHa544XZ5H7Z5hypmRQOBeQrp9ACDb2jGKuQ==;"

In [21]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION
os.environ["AZURE_SEARCH_KEY"] = AZURE_SEARCH_KEY
os.environ["AZURE_SEARCH_ENDPOINT"] = AZURE_SEARCH_ENDPOINT
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["AZURE_COSMOSDB_ENDPOINT"]=  AZURE_COSMOSDB_ENDPOINT
os.environ["AZURE_COSMOSDB_NAME"]=  AZURE_COSMOSDB_NAME
os.environ["AZURE_COSMOSDB_CONTAINER_NAME"]=  AZURE_COSMOSDB_CONTAINER_NAME
os.environ["AZURE_COMOSDB_CONNECTION_STRING"]=  AZURE_COMOSDB_CONNECTION_STRING

### Let's start with the basics
Let's use a very simple example to see if the GPT model of Azure OpenAI have memory. We again will be using langchain to simplify our code 

In [22]:
QUESTION = "Tell me some use cases for reinforcement learning?"
FOLLOW_UP_QUESTION = "Can you rephrase what you just said?"

In [23]:
# Define model
MODEL = "gpt-35-turbo"
# Create an OpenAI instance
llm = AzureChatOpenAI(deployment_name=MODEL, temperature=0.5)

In [24]:
# We create a very simple prompt template, just the question as is:
prompt = PromptTemplate(
    input_variables=["question"],
    template="{question}",
)

chain = LLMChain(llm=llm, prompt=prompt)

In [25]:
# Let's see what the GPT model responds
response = chain.run(QUESTION)
printmd(response)

1. Game playing: Reinforcement learning has been successfully applied to play games such as chess, Go, and poker, where the agent learns to make optimal moves by trial and error.

2. Robotics: Reinforcement learning is used to train robots to perform tasks such as grasping objects, navigating through environments, and manipulating objects.

3. Autonomous driving: Reinforcement learning can be used to train self-driving cars to make optimal decisions in real-world scenarios, such as navigating through traffic and avoiding obstacles.

4. Recommendation systems: Reinforcement learning can be used to develop personalized recommendation systems that learn from user behavior and feedback.

5. Advertising: Reinforcement learning can be used to optimize online advertising campaigns by learning which ads perform best and adjusting the strategy accordingly.

6. Energy management: Reinforcement learning can be used to optimize energy consumption in buildings and other systems by learning to balance energy usage with occupant comfort and cost.

7. Healthcare: Reinforcement learning can be used to develop personalized treatment plans for patients based on their medical history and response to treatment. 

8. Financial trading: Reinforcement learning can be used to develop trading algorithms that learn to make optimal decisions based on market conditions and historical data.

In [26]:
#Now let's ask a follow up question
chain.run(FOLLOW_UP_QUESTION)

'As an AI language model, I cannot recall what I have said previously unless you provide me with the specific statement you want me to rephrase. Please provide me with more context or information.'

As you can see, it doesn't remember what it just responded. This proof that the LLM does NOT have memory and that we need to give the memory as a a conversation history as part of the prompt, like this:

In [27]:
hist_prompt = PromptTemplate(
    input_variables=["history", "question"],
    template="""
                {history}
                Human: {question}
                AI:
            """
    )
chain = LLMChain(llm=llm, prompt=hist_prompt)

In [28]:
Conversation_history = """
Human: {question}
AI: {response}
""".format(question=QUESTION, response=response)

In [29]:
chain.run({"history":Conversation_history, "question": FOLLOW_UP_QUESTION})

'Certainly! Some examples of where reinforcement learning can be applied include game playing, robotics, autonomous driving, recommendation systems, advertising, energy management, healthcare, and financial trading. In these scenarios, the agent learns to make optimal decisions through trial and error, based on feedback received from the environment.'

**Bingo!**, so we now know how to create a chatbot using LLMs, we just need to keep the state/history of the conversation and pass it as context every time

## Now that we understand the concept of memory via adding history as a context, let's go back to our GPT Smart Search engine

In order to not duplicate code, we have put many of the code used in Notebook 3 into functions. These functions are in the app/utils.py and app/prompts.py files This way we can use these functios in the app that we will build later.

In [30]:
index1_name = "cogsrch-index-files"
index2_name = "cogsrch-index-csv"
indexes = [index1_name, index2_name]

agg_search_results = get_search_results(QUESTION, indexes)
ordered_results = order_search_results(agg_search_results, reranker_threshold=1)

In [31]:
docs = []
for key,value in ordered_results.items():
    for page in value["chunks"]:
        docs.append(Document(page_content=page, metadata={"source": value["location"]}))

# Calculate number of tokens of our docs
tokens_limit = model_tokens_limit(MODEL)

if(len(docs)>0):
    num_tokens = num_tokens_from_docs(docs)
    print("Custom token limit for", MODEL, ":", tokens_limit)
    print("Combined docs tokens count:",num_tokens)
        
else:
    print("NO RESULTS FROM AZURE SEARCH")


Custom token limit for gpt-35-turbo : 3000
Combined docs tokens count: 111872


In [32]:
%%time
if num_tokens > tokens_limit:
    index = embed_docs(docs)
    top_docs = search_docs(index,QUESTION)
    
    # Now we need to recalculate the tokens count of the top results from similarity vector search
    # in order to select the chain type: stuff or map_reduce
    
    num_tokens = num_tokens_from_docs(top_docs)   
    print("Token count after similarity search:", num_tokens)
    chain_type = "map_reduce" if num_tokens > tokens_limit else "stuff"
    
else:
    # if total tokens is less than our limit, we don't need to vectorize and do similarity search
    top_docs = docs
    chain_type = "stuff"
    
print("Chain Type selected:", chain_type)

Number of chunks: 99
Token count after similarity search: 4728
Chain Type selected: map_reduce
CPU times: user 18.3 s, sys: 1.42 s, total: 19.7 s
Wall time: 5.68 s


In [34]:
# Get the answer
response = get_answer(docs=top_docs, query=QUESTION, language="English", deployment=MODEL, chain_type=chain_type)
response['output_text']

'Use cases for reinforcement learning include budgeted learning, the multi-armed bandit problem, and active model selection. Reinforcement learning is also used in applications such as routing. There is no specific answer to the question of use cases for reinforcement learning in the second source.\nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0604/0604010v2.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/9605/9605103v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf'

And if we ask the follow up question:

In [35]:
response = get_answer(docs=top_docs,  query=FOLLOW_UP_QUESTION, language="English",deployment=MODEL, chain_type=chain_type)
response['output_text']

'There is no relevant text in the given portion of the document that directly answers the question.\nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0604/0604010v2.pdf'

Until now we just have the same as the prior Notebook 03: results from Azure Search enhanced by OpenAI model, with no memory

**Now let's add memory to it:**

Reference: https://python.langchain.com/en/latest/modules/memory/examples/adding_memory_chain_multiple_inputs.html

In [36]:
# memory object, which is neccessary to track the inputs/outputs and hold a conversation.
memory = ConversationBufferMemory(memory_key="chat_history",input_key="question")

response = get_answer(docs=top_docs, query=QUESTION, language="English", deployment=MODEL, chain_type=chain_type, 
                        memory=memory)
response['output_text']

'Reinforcement learning has a variety of use cases, such as robotic reinforcement learning and routing. While there is no single document that lists all of these use cases, different techniques and approaches to reinforcement learning can be applied to various problems and applications. For example, problem decomposition can be used to solve complex reinforcement-learning problems. Additionally, some methods represent uncertainty as a simple additive factor to the normal expected reward estimates and acquire further meaning when viewed through a statistical decision-making framework. \n\nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0604/0604010v2.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/9605/9605103v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf'

In [37]:
# Now we add a follow up question:
response = get_answer(docs=top_docs, query=FOLLOW_UP_QUESTION, language="English", deployment=MODEL, chain_type=chain_type, 
                      memory=memory)
response['output_text']

'I apologize for the confusion. In my previous response, I mentioned that reinforcement learning has various use cases, such as robotic reinforcement learning and routing. However, there is no single document that lists all of these use cases. Different techniques and approaches can be applied to various problems and applications. Problem decomposition can be used to solve complex reinforcement-learning problems, and some methods represent uncertainty as a simple additive factor to the normal expected reward estimates and acquire further meaning when viewed through a statistical decision-making framework. Is there anything else you would like me to clarify or expand upon?\n\nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0604/0604010v2.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/9605/9605103v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf'

In [38]:
# Another follow up query
response = get_answer(docs=top_docs, query="Thank you", language="English", deployment=MODEL, chain_type=chain_type,  
                      memory=memory)
response['output_text']

"You're welcome! If you have any other questions or if there is anything else I can help you with, please let me know. \n\nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0604/0604010v2.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/9605/9605103v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf"

Let's check our memory to see that it's keeping the conversation

In [39]:
memory.buffer

"Human: Tell me some use cases for reinforcement learning?\nAI: Reinforcement learning has a variety of use cases, such as robotic reinforcement learning and routing. While there is no single document that lists all of these use cases, different techniques and approaches to reinforcement learning can be applied to various problems and applications. For example, problem decomposition can be used to solve complex reinforcement-learning problems. Additionally, some methods represent uncertainty as a simple additive factor to the normal expected reward estimates and acquire further meaning when viewed through a statistical decision-making framework. \n\nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0604/0604010v2.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/9605/9605103v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf\nHuman: Can you rephrase what you just said?\nAI: I apologize for the confusion. In my previous response, I mentioned 

## Using CosmosDB as persistent memory

In previous cell we have added local RAM memory to our chatbot. However, it is not persistent, it gets deleted once the app user's session is terminated. It is necessary then to use a Database for persistent storage of each of the bot user conversations, not only for Analytics and Auditing, but also if we wisg to provide recommendations. 

Here we will store the conversation history into CosmosDB for future auditing purpose.
We will use a class in LangChain use CosmosDBChatMessageHistory, see [HERE](https://python.langchain.com/en/latest/_modules/langchain/memory/chat_message_histories/cosmos_db.html)

In [41]:
# Create CosmosDB instance from langchain cosmos class.

SESSION_ID = '001'  # this isstring value, normally you need to provide the user session id. As an example I have provided '001' as a session id
USER_ID = 'dummy01' # this is string value, normally you need to provide the user id. As an example I have provided 'dummy01' as a user id

cosmos = CosmosDBChatMessageHistory(
    cosmos_endpoint=AZURE_COSMOSDB_ENDPOINT,
    cosmos_database=AZURE_COSMOSDB_NAME,
    cosmos_container=AZURE_COSMOSDB_CONTAINER_NAME,
    connection_string=AZURE_COMOSDB_CONNECTION_STRING,
    session_id=SESSION_ID,
    user_id=USER_ID
    )

# prepare the cosmosdb instance
cosmos.prepare_cosmos()

In [42]:
# Create or Memory Object
memory = ConversationBufferMemory(memory_key="chat_history",input_key="question",chat_memory=cosmos)

In [43]:
# Testing using our Question
response = get_answer(docs=top_docs, query=QUESTION, language="English", deployment=MODEL, chain_type=chain_type, 
                        memory=memory)
response['output_text']

'Reinforcement learning can be applied to various real-world applications such as robotic manipulation with immediate reward, routing, and a variety of small problems. While there is no single paragraph in the given portion of the document that directly lists use cases for reinforcement learning, the document discusses various techniques and approaches to solving reinforcement learning problems that can be useful in different scenarios. Sources: https://demodatasetsp.blob.core.windows.net/arxivcs/0604/0604010v2.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/9605/9605103v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf.'

In [44]:
# Now we add a follow up question:
response = get_answer(docs=top_docs, query=FOLLOW_UP_QUESTION, language="English", deployment=MODEL, chain_type=chain_type, 
                      memory=memory)
response['output_text']

'I apologize for any confusion. What I meant to convey is that there is no single paragraph in the given portion of the document that directly lists use cases for reinforcement learning. However, the document discusses various techniques and approaches to solving reinforcement learning problems that can be useful in different scenarios. If you have a specific use case in mind, I can try to find more information about it. Sources: https://demodatasetsp.blob.core.windows.net/arxivcs/0604/0604010v2.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/9605/9605103v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf.'

In [45]:
# Another follow up query
response = get_answer(docs=top_docs, query="Thank you", language="English", deployment=MODEL, chain_type=chain_type,  
                      memory=memory)
response['output_text']

"You're welcome! If you have any other questions or need more information, please don't hesitate to ask. Sources: https://demodatasetsp.blob.core.windows.net/arxivcs/0604/0604010v2.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/9605/9605103v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf."

Let's check our Azure CosmosDB to see the whole conversation


In [46]:
#load message from cosmosdb
cosmos.load_messages()
cosmos.messages

[HumanMessage(content='Tell me some use cases for reinforcement learning?', additional_kwargs={}, example=False),
 AIMessage(content='Reinforcement learning can be applied to various real-world applications such as robotic manipulation with immediate reward, routing, and a variety of small problems. While there is no single paragraph in the given portion of the document that directly lists use cases for reinforcement learning, the document discusses various techniques and approaches to solving reinforcement learning problems that can be useful in different scenarios. Sources: https://demodatasetsp.blob.core.windows.net/arxivcs/0604/0604010v2.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/9605/9605103v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf.', additional_kwargs={}, example=False),
 HumanMessage(content='Can you rephrase what you just said?', additional_kwargs={}, example=False),
 AIMessage(content='I apologize for any confusion. What I mea

![CosmosDB Memory](./images/cosmos-chathistory.png)

# Summary
##### Adding memory to our application allows the user to have a conversation, however this feature is not something that comes with the LLM, but instead, memory is something that we must provide to the LLM in form of context of the question.

We added persitent memory using CosmosDB.

We also can notice that the current chain that we are using is smart, but not that much. Although we have given memory to it, it searches for similar docs everytime, it struggles to respond to prompts like: Hello, Thank you, Bye, What's your name, What's the weather and any other task that is not search in the knowledge base.



# NEXT
We know now how to do a Smart Search Engine that can power a chatbot!! great!

But, does this solve all the possible scenarios that a virtual assistant will require?  **What about if the answer to the Smart Search Engine is not related to text, but instead requires to look into tabular data?** The next notebook 05 explains and solves the tabular problem and the concept of Agents